Desafio 03 - Comparação RN e CNN <br>
Aluna: Giovanna de Sá Tabosa <br>
Classificar um dataset e comparar a diferença de performance dos modelos ao usar as abordagens de ML e DL no mesmo problema.


# Importação das bibliotecas necessárias

In [ ]:
#Conexão com o drive
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow.keras as keras
import pandas as pd

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical


# Carregamento, verificação e transformação dos dados

Foi utilizado o conjunto de dados **CIFAR-10** retirado do site kaggle, o dataset contém 60.000 imagens coloridas em 10 classes diferentes (avião, carro, pássaro, gato, etc.), com 6.000 imagens por classe.

In [ ]:
#Carregmento do conjunto de dados
train_df = pd.read_csv("/content/drive/My Drive/dados_mba/dataset/train.csv")
valid_df = pd.read_csv("/content/drive/My Drive/dados_mba/dataset/valid.csv")

#Carregando o módulo do conjunto de dados Keras para CIFAR-10:
from tensorflow.keras.datasets import cifar10

#Divididos entre conjuntos de treinamento e validação
(X_train, y_train), (x_valid, y_valid) = cifar10.load_data()

In [ ]:
#Verificação dos dados de treinamento
train_df.head()

In [ ]:
#Verificação dos dados de validação
valid_df.head()

# Resumindo os dados de treinamento e validação

In [ ]:
X_train.shape

(50000, 32, 32, 3)

50000: Número de imagens de treinamento <br> (32,32,3): dimensão da imagem 3272

In [ ]:
y_train.shape

(50000, 1)

50000: Número de rótulos de treinamento (um para cada imagem)<br>
1: Cada rótulo é uma única entrada (a classe da imagem)

In [ ]:
x_valid.shape

(10000, 32, 32, 3)

10000: Número de imagens de validação <br>
(32,32,3): dimensão da imagem 3272

In [ ]:
y_valid.shape

(10000, 1)

10000: Número de rótulos de teste <br>
1: Cada rótulo é uma única entrada


# Extração das imagens

In [ ]:
#Armazenamendo das imagens de treinamento e validação nas variáveis
x_train = train_df.values
x_valid = valid_df.values

# Categorizando as imagens

In [ ]:
#Carregamento do dataset CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
num_classes = 24

#Conversão dos rótulos para uma codificação categórica
y_train_categorical = to_categorical(y_train, 10)
y_test_categorical = to_categorical(y_test, 10)


# Visualizando os Dados
Para a vizualização, foi preciso remodelar os dados de sua forma atual de 3072 pixels para uma forma 2D de 32x32x3 pixels para dar sentido à imagem e também redimensioná-la.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(40, 40))

num_images = 20
for i in range(num_images):
    row = x_train[i]
    label = y_train[i]

    #No Dataset usado, as imagens têm 32x32x3
    #Usando os 3072 elementos e redimensionando
    image = row[:3072].reshape(32,32,3)
    plt.subplot(1, num_images, i+1)
    plt.title(label, fontdict={'fontsize': 30})
    plt.axis('off')
    plt.imshow(image)
    plt.show()


# Normalização dos dados da imagem

In [ ]:
x_train.min()

0

In [ ]:
x_train.max()

255

In [ ]:
#Normalização os dados (Valor max=255)
x_train = x_train / 255
x_valid = x_valid / 255

In [ ]:
x_train.min()

0.0

In [ ]:
x_train.max()

1.0

In [ ]:
#Excluindo coluna extra do conjunto treinamento x_train que apresenta 3073 colunas
x_train = x_train[:, :3072]


# Construção o modelo

In [ ]:
#Instanciando o modelo
from tensorflow.keras.models import Sequential

model = Sequential()

In [ ]:
#Criando os dados de entrada
from tensorflow.keras.layers import Dense

In [ ]:
model.add(Dense(units=512, activation='relu', input_shape=(3072,)))

In [ ]:
model.add(Dense(units = 512, activation='relu'))

In [ ]:
#Criando os dados de saída
model.add(Dense(units = 10, activation='softmax'))

In [ ]:
#Verificando os dados de treinamento e validação
print(x_train.shape)
print(x_valid.shape)


(50000, 3072)
(10000, 3072)


In [ ]:
#Resumindo o modelo
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_13 (Dense)                     │ (None, 512)                 │       1,573,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,841,162 (7.02 MB)

 Trainable params: 1,841,162 (7.02 MB)

 Non-trainable params: 0 (0.00 B)

Total de parâmetros:  1.841.162 <br>
Parâmetros treináveis: 1.841.162<br>
Parâmetros não treináveis: 0 <br>

In [ ]:
#Compilando o modelo
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
#Treinando o modelo
history = model.fit(
    x_train, y_train, epochs=5, verbose=1, validation_data=(x_valid, y_valid)
)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 37s 23ms/step - accuracy: 0.2573 - loss: 2.1512 - val_accuracy: 0.3649 - val_loss: 1.7594
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 40s 23ms/step - accuracy: 0.3810 - loss: 1.7364 - val_accuracy: 0.4298 - val_loss: 1.6178
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 40s 23ms/step - accuracy: 0.4117 - loss: 1.6419 - val_accuracy: 0.3599 - val_loss: 1.8473
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 44s 24ms/step - accuracy: 0.4307 - loss: 1.5905 - val_accuracy: 0.3309 - val_loss: 1.9929
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.4413 - loss: 1.5708 - val_accuracy: 0.4223 - val_loss: 1.6427


**Treinamento:**<br>
Começou com uma precisão de 25,73% na primeira época e aumentou para 44,13% na quinta época. <br>
**Validação:**<br>
A precisão no conjunto de validação começou em 36,49% e alcançou 42,43% na quinta época. <br>


# Conclusão

O modelo está aprendendo ao longo das épocas, com um aumento na acurácia e uma diminuição na perda. <br>
No entanto, o valor da acurácia do modelo (precisão) é baixo, não superando 50%. <br> Pode-se concluir que o modelo não é tão preciso para detectar e classificar as imagens.